In [90]:
import re

In [91]:
from collections import defaultdict

In [92]:
from pymystem3 import Mystem
stem = Mystem()
import json
from math import log

In [93]:
import requests
from bs4 import BeautifulSoup
from IPython.display import HTML, display
from collections import defaultdict
import os
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

stop_words = stopwords.words('russian') + ['И', 'и', 'К','За','к', 'за','а','но','в','на','под','А','Под', 'Но', 'В','На']
toktok = ToktokTokenizer()

## ДЗ 

Написать краулер, который собирает тексты с новостного ресурса и выдает список релевантных документов по запросу

In [94]:
def get_main_links(kaz_url):
    req = requests.get(kaz_url)
    soup = BeautifulSoup(req.text, 'lxml')
    main_links = []
    for i in soup.findAll('li',attrs = {'class':['leaf', 'leaf first']}):
        if 'categories' in i.contents[0]['href']:
            main_links.append(('http://www.evening-kazan.ru'+ i.contents[0]['href']+'?page=', i.text))
    return main_links

In [42]:
# kaz_url = 'http://www.evening-kazan.ru'

In [43]:
# req = requests.get(kaz_url)

In [44]:
# soup = BeautifulSoup(req.text, 'lxml')

In [45]:
# main_links = []
# for i in soup.findAll('li',attrs = {'class':['leaf', 'leaf first']}):
#     if 'categories' in i.contents[0]['href']:
#         main_links.append(('http://www.evening-kazan.ru'+ i.contents[0]['href']+'?page=', i.text))

In [95]:
main_links = get_main_links('http://www.evening-kazan.ru')

In [96]:
def get_links_for_texts(main_links):
    count_all = 0
    links_from_class = set()
    for class_link in main_links:
        topic = class_link[1]
        for i in range(4):
            try:
                req = requests.get(class_link[0]+str(i))
                soup = BeautifulSoup(req.text, 'lxml')
                for i in soup.findAll('a', href=True):
                    if 'http://www.evening-kazan.ru/articles' in i['href'] and '#disqus_thread' not in i['href'] and '#comment'not in i['href']:
                        links_from_class.add((i['href'], topic))
                        count_all+=1
            except:
                break
    return links_from_class

# " тут мы получили ссылки на чуть более чем 1к статей"

In [97]:
links_from_class = get_links_for_texts(main_links)

In [98]:
#get_texts_and_info

def get_corpus(links_from_class):
    lens_of_texts = defaultdict(int)
    index = 1
    for link in links_from_class:
        try:
            req = requests.get(link[0])
            soup = BeautifulSoup(req.text, 'lxml')
            author = soup.findAll('div', attrs = {'class':'author heading--meta'})[0].text
            title = soup.findAll('h1', attrs = {'class':'title title-story'})[0].text
            date = soup.findAll('div', attrs = {'submitted heading--meta'})[0].text
            text = soup.findAll('div', attrs = {'class':'node'})
            text_to_write = ''
            for i in text:
                for item in i.findAll('p'):
                    text_to_write+=item.text.lower()
            with open('/Users/Valeriya/Infopoisk/corpus/'+str(index)+'.txt', 'w') as file:
                file.write('@au '+author+'\n'+'@ti '+title+'\n'+'@da '+date+'\n'+'@topic '+link[1]+'\n'+'@url '+link[0]+'\n'+text_to_write)
                lens_of_texts[str(index)+'.txt'] = len(toktok.tokenize(text_to_write))
            index+=1
        except Exception as e:
            print(e)

In [99]:
k1 = 2.0
b = 0.75

def score_BM25(n, qf, N, dl, avdl):
    K = compute_K(dl, avdl)
    IDF = log((N - n + 0.5) / (n + 0.5))
    frac = ((k1 + 1) * qf) / (K + qf)
    return IDF * frac


def compute_K(dl, avdl):
    return k1 * ((1-b) + b * (float(dl)/float(avdl)))

In [100]:
def read_tokenize_and_get_inverse_index_txt(path):
    n_texts = 0
    sum_lens = 0
    matrix = defaultdict(dict)
#     doc_num = 0
    for root, dirs, files in os.walk(path):
        for filename in files:
            open_name = os.path.join(root, filename)
#             doc_num += 1
#             print(open_name)
            with open(open_name, 'r', encoding='utf-8', errors='ignore') as f:
                text = f.read().lower()
                n_texts +=1
                sum_lens += len(toktok.tokenize(text))
                tokens = [word for word in toktok.tokenize(text) if word not in stop_words]
                for word in set(tokens):
                    count_of_word = tokens.count(word)
                    lemma = stem.lemmatize(word)[0]
                    matrix[lemma][filename] = count_of_word # либо add(docnum), если хочется просто номера док-тов
    avlen = sum_lens/n_texts
    return matrix, avlen, n_texts

In [101]:
getting_inverse_etc = read_tokenize_and_get_inverse_index_txt('/Users/Valeriya/Infopoisk/corpus')

In [102]:
inverted_inds = getting_inverse_etc[0]
avdl = getting_inverse_etc[1]
# avdl

In [103]:
with open('inverse_inds.txt', 'w',encoding='utf-8') as outfile:
    json.dump(inverted_inds, outfile, ensure_ascii = False)
# записала для того, чтоб в приложении на Flask потом не пересчитывать

In [104]:
with open('lens.txt', 'w',encoding='utf-8') as outfile:
    json.dump(lens_of_texts, outfile, ensure_ascii = False)

# записала для того, чтоб в приложении на Flask потом не пересчитывать

In [105]:
Q = 'каникулы на новый год и рождество'.split()

In [106]:
def get_relevance_list(Q):
    N = 1151
    relevance_list = defaultdict(int)
    for word in Q:
        if word not in stop_words:
            word = word.lower()
            q = stem.lemmatize(word)[0]
            for doc_name in inverted_inds[q]:
                n = len(inverted_inds[q])
                qf = inverted_inds[q][doc_name]
                if 'ipynb' not in doc_name:
                    text = open('/Users/Valeriya/Infopoisk/corpus/'+doc_name, 'r')
                    read_txt = text.read()
                    index = score_BM25(n, qf, N, lens_of_texts[doc_name], avdl)
                    link = re.findall('@url(.+)', read_txt)[0]
        #             print(link)
        #             break
                    relevance_list[link] += index
    return relevance_list

In [107]:
relevance_list = get_relevance_list(Q)

In [108]:
for link in sorted(relevance_list.items(), key=lambda x: x[1], reverse=True)[:10]:
    display(link[0])

' http://www.evening-kazan.ru/articles/gripp-v-kazan-pridet-posle-deda-moroza.html'

' http://www.evening-kazan.ru/articles/po-posledney-mile-kazancy-poydut-peshkom.html'

' http://www.evening-kazan.ru/articles/desyatiletnego-kazanskogo-shkolnika-ubilo-tokom-v-cerkvi.html'

' http://www.evening-kazan.ru/articles/v-trogatelnyh-zooparkah-kazani-rabotayut-zveri-nelegaly.html'

' http://www.evening-kazan.ru/articles/uchitelyam-v-tatarstane-snova-ustroili-proverku.html'

' http://www.evening-kazan.ru/articles/letom-umneem-zimoy-tupeem.html'

' http://www.evening-kazan.ru/articles/shkolnikov-v-tatarstane-zaedayut-vshi.html'

' http://www.evening-kazan.ru/articles/uchitelya-v-tatarstane-otkazyvayutsya-sdavat-ege-i-prosyat-ministra-pokazat-primer.html'

' http://www.evening-kazan.ru/articles/meru-kazani-pozhalovalis-na-adskie-usloviya-dlya-medikov-v-detskih-lageryah.html'

' http://www.evening-kazan.ru/articles/otoplenie-v-kazani-vklyuchat-posle-potepleniya.html'